# Delay discounting analyis: SOLO models
This notebooks gives an overview of using SOLO models for analysing delay discounting data. The SOLO models estimate parameters for each data file independently from the rest. Further, each datafile is processed entirely separately. This is scalable, thus useful for very large datasets. We avoid building _very_ large models with 100's or 1,000's or participants. It can still take time, but the point is we avoid both memory and computational capacity limitations.

## Parameter estimation
We can do parameter estimation by creating a model instance and calling the `sample_posterior` method while providing the data.

## Posterior prediction
Once we have a posterior distribution over the parameters given the data, then we can do some posterior predictive model checking by plotting the predicted discount function along with the data. This is done with the `plot_discount_functions_region` method.
However we can also use the `df_comparison(models, data)` function in order to plot the data along with posterior predictions of mulitple methods.

## Model evaluation
Some qualitative or sanity-check model evaluation is done with plotting the posterior predictions (see above). However, we might also want to do some quantitative evaluation.

- WAIC
- LOO

We also calculate the log loss goodness of fit metric.

First, some basic boilerplate setup code

In [ ]:
# file handling
from glob import glob
import os

# data + modelling
import math
import numpy as np
import pandas as pd
import pymc3 as pm

# plotting
import seaborn as sns
%config InlineBackend.figure_format = 'retina'


import matplotlib.pyplot as plt

# set up plotting preferences
plt.style.use('seaborn-darkgrid')

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# comment out to avoid plotting MANY figures, bloating the notebook
# %matplotlib inline
# %matplotlib notebook

## Import toolbox code

In [ ]:
# autoreload imported modules. Convenient while I'm developing the code.
%load_ext autoreload
%autoreload 2

In [ ]:
from models_solo import *
from df_data import *
#from df_plotting import df_comparison
from model_comparison import *
from fitting import fit

# Data import
The approach we will take is to have two dataframes:
- experiment level data. Each row will correspond to one datafile, and will have columns including the filename, but also experiment level information such as condition, or other participant and experiment data.
- all response data. This will be one large dataframe which includes _all_ raw response data, all trials for all participants.

## Data preparation step 1: Construct experiment-level information

In [ ]:
files = glob('data/non_parametric/*.txt')
files

Start to build the experiment level dataframe

In [ ]:
expt_data = pd.DataFrame({'filename': files})
expt_data

Now we will parse the filenames to extract relevant information. This is good when you have put this information into the filenames. But in some situations you might have this experiment level information in a separate file. In that case, you just want to import it.

In [ ]:
def parse_filename(fname):
    """Extract initials from provided filename"""
    path, file = os.path.split(fname)
    initials = file.split('-')[0]
    return initials

expt_data['id'] = pd.Series([parse_filename(fname) for fname in files],
                           index=expt_data.index)
expt_data

## Data preparation step 2: Import raw behavioural data
The goal here is to import a list of user-specified raw data files and bundle them up into a pandas dataframe.

The approach taken here is to keep this raw code here in the notebook so that the data import process is transparent and easily modifiable by users if their data is in slightly different forms.

In [ ]:
# We already have a list of filenames, but we can extract from expt_data
filenames = expt_data['filename']

raw_data = import_raw_data(filenames)
raw_data.head()

If we like, we can extract list of unique participant names:

    participant_list = list(expt_data.id.unique())
    print(participant_list)

# Parameter estimation + Model Comparison

First we set up a list of models that we want to examine. Then we 'fit' these models (parameter estimation) then do Bayesian model comparison.

In [ ]:
models = [Coinflip,
          Exponential, 
          Hyperbolic,
          HyperboloidA, 
          HyperboloidB, 
          ConstantSensitivity, 
          ExponentialPower,
          ExponentialLog,
          HyperbolicLog,
          DoubleExponential,
          BetaDelta,
          TradeOff,
          ITCH,
          DRIFT]

# When we do model comparison we want model names in the WAIC/LOO plots. There will be a better solution, but we currenrly implement the workaround.
# See https://discourse.pymc.io/t/can-we-add-model-names-when-we-do-model-comparison/935/2 for more.
MODEL_NAME_MAP = {
    0: "Coinflip",
    1: "Exponential",
    2: "Hyperbolic",
    3: "Hyperboloid A",
    4: "Hyperboloid B",
    5: "Constant Sensitivity",
    6: "Exponential Power",
    7: "Exponential Log",
    8: "Hyperbolic Log",
    9: "Double Exponential",
    10: "BetaDelta",
    11: "TradeOff",
    12: "ITCH",
    13: "DRIFT"
}

In [ ]:
# models = [Coinflip,
#           Exponential, 
#           Hyperbolic]

# # When we do model comparison we want model names in the WAIC/LOO plots. There will be a better solution, but we currenrly implement the workaround.
# # See https://discourse.pymc.io/t/can-we-add-model-names-when-we-do-model-comparison/935/2 for more.
# MODEL_NAME_MAP = {
#     0: "Coinflip",
#     1: "Exponential",
#     2: "Hyperbolic"
# }

In [ ]:
# NOTE: This will take time to compute! And it will save outputs to the specified directory
results = fit(models, raw_data, expt_data, MODEL_NAME_MAP, save_dir='temp_analysis')

We should now have a series of saved plots. These are all located in the specified `save_dir` which by default equals `'temp'`. In this folder there are model comparison plots, one for each participant. There are also subfolders for each participant, which contains a series of plots for model diagnostics etc.

**TODO list for fit function**

- return model fits from the `fit` function OR save them to disk